In [115]:
import numpy as np
import math
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

In [ ]:
mod = SourceModule("""
    #define N 1000
    __device__ float f(float x)
    {	
        float mu=1.0;
        float lambda=1.0;
        float fx;
        if(x>0.01){
            fx=(sqrt(lambda/(2*M_PI*pow(x,3))))*exp(((-1*lambda*pow((x-mu),2))/(2*pow(mu,2)*x)));
        }
            else
            fx=0;

        return fx;

    }
    
    __device__ float integral(float x1, float x2, float x3, float x4, float x5){
            //float x1=0,x2=0,x3=0,x4=0; /*these are the main variable, use range (-2:0.1:2) */
            float i, a=0.0, b=50, sum=0;
            /* printf("nThis program will integrate a function between two boundary limits."); */
             if (a > b) {
              i = a;
              a = b;
              b = i;
             }
             for (i = a; i < b; i += (b - a) / N) {
              float y;
              y = f(i)*f(i-x1)*f(i-x2)*f(i-x3)*f(i-x4)*f(i-x5);
              //printf("%f",y);
              sum += y * (b - a) / N;
             }
            return sum;
    }

    
    
    __global__ void doSome(float *in, int total, int d )
    { 
       
        int idx = blockIdx.x * blockDim.x + threadIdx.x; 

        if(idx >= total){
            return;
        }
        //printf ("%d:::::::\\t", idx);
       
        in[idx*(d+1)+5]=integral(in[idx*(d+1)+0],in[idx*(d+1)+1], in[idx*(d+1)+2], in[idx*(d+1)+3], in[idx*(d+1)+4]);
    }""")


In [ ]:
func = mod.get_function("doSome")
d_in = cuda.mem_alloc(h_in.nbytes)
cuda.memcpy_htod(d_in, h_in)

In [88]:
Z = np.random.randn(25921,3)
Z = np.abs(Z)

In [89]:
def in_dist(a,b,tau=1):
    ps = np.empty(len(a))
    for i in range(len(a)):
        if(b[i] >= a[i]):
            ps[i] = (1/tau**2)*(math.exp(-1*b[i]/tau))
        else: 
            ps[i] = 0
    return ps
            

In [101]:
x, y = np.meshgrid([x for x in np.arange(0,2.05,0.05)],[x for x in np.arange(0,2.05,0.05)])

In [102]:
t = y 
y = np.reshape(x,-1)
x =np.reshape(t,-1)

In [103]:
x

array([ 0.,  0.,  0., ...,  2.,  2.,  2.])

In [104]:
y

array([ 0.  ,  0.05,  0.1 , ...,  1.9 ,  1.95,  2.  ])

In [105]:
pin = in_dist(x,y, 0.5)
pin = pin.T

In [106]:
pin

array([ 4.        ,  3.61934967,  3.27492301, ...,  0.        ,
        0.        ,  0.07326256])

In [107]:
indexy = np.array([],dtype=int)
for k in range (1,42):
    
    a = [i for i in range(1+161*(k-1),42+161*(k-1))] ### possible index error
    
    indexy =np.append(indexy,a)


In [108]:
indexx = np.array([],dtype=int)
for k in range (0,121):
    
    a = [i for i in range(161*(k-1),121+161*(k-1))] ### possible index error
    
    indexx =np.append(indexx,a)

In [109]:
PYX = np.empty(len(indexy))
PY = np.empty(len(indexx))

In [110]:
count = 0
for j in indexy:
    A = Z[indexx+j,2]
    PYX [count] = 0.05*0.05*np.sum(A*np.log(A))
    count = count + 1

In [111]:
count = 0
for j in indexx:
    B = Z[indexy+j,2]
    PY [count] = 0.05*0.05*np.sum(B*pin)
    count = count + 1

In [112]:
PYX

array([ 1.73605918,  1.77749142,  1.76467234, ...,  1.6933896 ,
        1.70473901,  1.67658168])

In [113]:
PY

array([ 0.79149454,  0.7814128 ,  0.78743315, ...,  0.76405792,
        0.76662481,  0.76792314])